[![imagenes](imagenes/pythonista.png)](https://pythonista.io)

In [ ]:
from flask import Flask, jsonify, request, abort
from data import campos, orden
from json import loads
from flask_sqlalchemy import SQLAlchemy

In [ ]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///data/alumnos.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

In [ ]:
class Alumno(db.Model):
    __tablename__ = 'alumnos'
    cuenta = db.Column(db.Integer, primary_key=True)
    nombre = db.Column(db.String(50))
    primer_apellido = db.Column(db.String(50))
    segundo_apellido = db.Column(db.String(50))
    carrera = db.Column(db.String(50))
    semestre = db.Column(db.Integer)
    promedio = db.Column(db.Float)
    al_corriente = db.Column(db.Boolean)

In [ ]:
def es_tipo(dato, tipo):
    if tipo == str:
        return True
    else:
        try: 
            return tipo(dato) is dato
        except:
            return False
        
def reglas(dato, campo):
    from data import carreras
    if campo == "Carrera" and dato not in carreras:
        return False
    elif campo == "Semestre" and dato < 1:
        return False
    elif campo == "Promedio" and (dato < 0 or dato > 10):
        return False
    elif (campo in ("Nombre", "Primer Apellido") and (dato == "")):
        return False
    else:
        return True           
    
def valida(dato, campo):
    return es_tipo(dato, campos[campo][0]) and reglas(dato, campo)

In [ ]:
def convierte(cuenta, alumno, orden):
    respuesta = {'cuenta': cuenta}
    for campo in orden:
        respuesta[campo] = getattr(alumno, campo.lower().replace(' ', '_'))
    return respuesta

In [ ]:
def recurso_completo(cuenta, peticion):
    try:
        candidato = Alumno(cuenta=int(cuenta))
        peticion = loads(peticion)
        if (set(peticion)).issubset(set(orden)):                    
            for campo in orden:
                if not campos[campo][1] and campo not in peticion:
                    setattr(candidato, campo.lower().replace(' ', '_'), '')
                elif valida(peticion[campo], campo):
                    setattr(candidato, campo.lower().replace(' ', '_'), peticion[campo])    
                else:
                    abort(400)
        else:
            abort(400)
    except:
        abort(400)
    db.session.add(candidato)
    db.session.commit()                        
    return jsonify(convierte(cuenta, candidato, orden))

In [14]:
@app.route('/api/<cuenta>', methods=['GET', 'POST', 'PUT', 'PATCH', 'DELETE'])
def api(cuenta):
    
    if request.method == 'GET':
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno == None:
            abort(404)
        else:
            return jsonify(convierte(cuenta, alumno, orden))
            
            
    if request.method == 'DELETE':
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno == None:
            abort(404)
        else:
            db.session.delete(alumno)
            db.session.commit()
            return jsonify(convierte(cuenta, alumno, orden))
        
        
    if request.method == 'POST':
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno == None:
            return recurso_completo(cuenta, request.data)
        else:
            abort(409)
            
            
    if request.method == 'PUT':
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno == None:
            abort(404)
        else:
            db.session.delete(alumno)
            db.session.commit()
            return recurso_completo(cuenta, request.data)

    if request.method == 'PATCH':               
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno == None:
            abort(404)
        else:
            try:
                peticion = loads(request.data)
                if (set(peticion)).issubset(set(orden)):
                    for campo in peticion:
                        dato = peticion[campo]
                        if valida(dato, campo):
                            setattr(alumno, campo.lower().replace(' ', '_'), dato)
                        else:
                            abort(400)
                else:
                    abort(400)
            except:
                abort(400)
            db.session.add(alumno)
            db.session.commit()                        
            return jsonify(convierte(cuenta, alumno, orden))
app.run('0.0.0.0')

In [16]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [17]:
%sql sqlite:///data/alumnos.db

'Connected: @data/alumnos.db'

In [18]:
%%sql 
select * from alumnos

 * sqlite:///data/alumnos.db
Done.


cuenta,nombre,primer_apellido,segundo_apellido,carrera,semestre,promedio,al_corriente
1231221,Pedro,Solis,Cabañas,Arquitectura,3,7.8,0
1231222,Yolanda,Jiménez,Lerdo,Actuaría,3,6.0,0
1231223,Juan,Ramos,Breña,Sistemas,9,8.6,1
1231224,Mayra Jimena,Cervantes,Lisama,Derecho,12,9.2,1


<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>